In [18]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

# Experimento 7: 
- LGBM
- Uso de DWT
- Estandarizacion del target
- Usando funcion entrenamiento: semillerio_en_prediccion
- Mismas variables
- Pesos: (log o max?)
- sqlite:///optuna_studies_v16.db
- Kaggle =  


##### Levantamos el dataset con target ya calculado

In [2]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado_201912.csv", sep=',', encoding='utf-8')
print("Dataset sin transformar tenia esto: (31362, 19)")
df.shape

Dataset sin transformar tenia esto: (31362, 19)


(31362, 35)

In [ ]:
import pandas as pd
import numpy as np
from dtaidistance import dtw
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt

# ----------------------------
# Paso 1: cargar y preparar datos
# ----------------------------

# Supongamos que tu DataFrame tiene: 'item_id', 'timestamp', 'target'
# y querés una serie por 'item_id'
# MUY IMPORTANTE: primero agrupar o muestrear para reducir tamaño
df = pd.read_csv('tus_series.csv')  # o cargá tu df directamente

# Por ejemplo: convertir a mensual y tomar promedio
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp').groupby('item_id').resample('M')['target'].mean().reset_index()

# Pivotear a formato wide (una serie por fila)
series_wide = df.pivot(index='item_id', columns='timestamp', values='target')

# Opcional: rellenar NaNs si hay pocos
series_wide = series_wide.fillna(method='ffill', axis=1).fillna(method='bfill', axis=1)

# ----------------------------
# Paso 2: calcular matriz de distancias DTW
# ----------------------------

# Convertir cada fila a lista para calcular DTW
series_list = series_wide.values.tolist()

# Usar fast DTW si querés velocidad: (solo para longitudes grandes)
from dtaidistance import dtw_distance_matrix_fast
distance_matrix = dtw_distance_matrix_fast(series_list, parallel=True)

# ----------------------------
# Paso 3: clustering jerárquico
# ----------------------------

# linkage requiere una forma "condensed" de la matriz
from scipy.spatial.distance import squareform

linkage_matrix = linkage(squareform(distance_matrix), method='ward')

# ----------------------------
# Paso 4: visualización o etiquetas
# ----------------------------

# Dendrograma
plt.figure(figsize=(12, 5))
dendrogram(linkage_matrix, labels=series_wide.index.tolist(), leaf_rotation=90)
plt.title('Clustering jerárquico con DTW')
plt.show()

# Obtener etiquetas de clúster
n_clusters = 4
labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# Añadir al dataframe original
resultado = pd.DataFrame({
    'item_id': series_wide.index,
    'cluster': labels
})
